<a href="https://colab.research.google.com/github/npmOSU/BloomaloRepo/blob/main/20thFeb2023/FinalSemanticSearch(PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semantic search with FAISS (PyTorch)

Installing necessary libraries for execution

In [2]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 13.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: 

Loading The CSV dataset

In [4]:
from datasets import load_dataset
issues_dataset = load_dataset("csv", data_files="EmbExtractedText.csv",encoding='cp1252',split="train")
issues_dataset

Dataset({
    features: ['title', 'body', 'comments', 'html_url'],
    num_rows: 478
})

In [5]:
# issues_dataset = issues_dataset.filter(
#     lambda x: (x["is_pull_request"] == False and len(x["comments"]) > 0)
# )
# issues_dataset

We renamed the columns to the following, this we for conveience of code execution. Will rename in the coming days. 

*   Title=title(Title of the article)
*   Subheading=body(Subheadings with each article)
*   Paragraph=comments(Content within each subparagraph)
*   URL=html_url(URL for the article)

In [6]:
columns = issues_dataset.column_names
columns_to_keep = ["title", "body", "html_url", "comments"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
issues_dataset = issues_dataset.remove_columns(columns_to_remove)
issues_dataset

Dataset({
    features: ['title', 'body', 'comments', 'html_url'],
    num_rows: 478
})

In [18]:
issues_dataset.set_format("pandas")
df = issues_dataset[:]
df

,title,body,comments,html_url
0,Anxiety Disorders (for Parents) - Nemours Kid...,What Are Anxiety Disorders?,Anxiety disorders cause extreme fear and worry...,https://kidshealth.org/en/parents/anxiety-diso...
1,Anxiety Disorders (for Parents) - Nemours Kid...,What Are the Kinds of Anxiety Disorders?,Different anxiety disorders can affect kids an...,https://kidshealth.org/en/parents/anxiety-diso...
2,Anxiety Disorders (for Parents) - Nemours Kid...,What Are the Signs & Symptoms of Anxiety?,A parent or teacher may see signs that a child...,https://kidshealth.org/en/parents/anxiety-diso...
3,Anxiety Disorders (for Parents) - Nemours Kid...,What Causes Anxiety Disorders?,Several things play a role in causing the over...,https://kidshealth.org/en/parents/anxiety-diso...
4,Anxiety Disorders (for Parents) - Nemours Kid...,How Are Anxiety Disorders Diagnosed?,Anxiety disorders can be diagnosed by a traine...,https://kidshealth.org/en/parents/anxiety-diso...
...,...,...,...,...
473,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Toddler Milestones,20. “We blow bubbles to calm [them] down and r...,https://www.mother.ly/toddler/toddler-mileston...
474,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Viral & Trending,21. “I’ve learned it’s different for each chil...,https://www.mother.ly/toddler/toddler-mileston...
475,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Toddler Milestones,Related Stories,https://www.mother.ly/toddler/toddler-mileston...
476,23 Mom-Tested Ways to Manage a Tantrum - Motherly,We believe in creating a world in which all mo...,Contact Us P: Advertise with Us P: Share Your ...,https://www.mother.ly/toddler/toddler-mileston...


Here we noticed issue with the following row values, so we deleted these entries

In [19]:
l=[0,37,215,249,257,258]
df = df.drop(labels=l, axis=0)

df

,title,body,comments,html_url
1,Anxiety Disorders (for Parents) - Nemours Kid...,What Are the Kinds of Anxiety Disorders?,Different anxiety disorders can affect kids an...,https://kidshealth.org/en/parents/anxiety-diso...
2,Anxiety Disorders (for Parents) - Nemours Kid...,What Are the Signs & Symptoms of Anxiety?,A parent or teacher may see signs that a child...,https://kidshealth.org/en/parents/anxiety-diso...
3,Anxiety Disorders (for Parents) - Nemours Kid...,What Causes Anxiety Disorders?,Several things play a role in causing the over...,https://kidshealth.org/en/parents/anxiety-diso...
4,Anxiety Disorders (for Parents) - Nemours Kid...,How Are Anxiety Disorders Diagnosed?,Anxiety disorders can be diagnosed by a traine...,https://kidshealth.org/en/parents/anxiety-diso...
5,Anxiety Disorders (for Parents) - Nemours Kid...,How Are Anxiety Disorders Treated?,"Most often, anxiety disorders are treated with...",https://kidshealth.org/en/parents/anxiety-diso...
...,...,...,...,...
473,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Toddler Milestones,20. “We blow bubbles to calm [them] down and r...,https://www.mother.ly/toddler/toddler-mileston...
474,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Viral & Trending,21. “I’ve learned it’s different for each chil...,https://www.mother.ly/toddler/toddler-mileston...
475,23 Mom-Tested Ways to Manage a Tantrum - Motherly,Toddler Milestones,Related Stories,https://www.mother.ly/toddler/toddler-mileston...
476,23 Mom-Tested Ways to Manage a Tantrum - Motherly,We believe in creating a world in which all mo...,Contact Us P: Advertise with Us P: Share Your ...,https://www.mother.ly/toddler/toddler-mileston...


In [8]:
# df["comments"][0].tolist()

In [9]:
# comments_df = df.explode("comments", ignore_index=True)
# comments_df.head(4)

In [20]:
from datasets import Dataset

comments_dataset = Dataset.from_pandas(df)
comments_dataset

Dataset({
    features: ['title', 'body', 'comments', 'html_url', '__index_level_0__'],
    num_rows: 472
})

In [14]:
# comments_dataset = comments_dataset.map(
#     lambda x: {"comment_length": len(x["comments"].split())}
# )

In [15]:
# comments_dataset = comments_dataset.filter(lambda x: x["comment_length"] > 15)
# comments_dataset

In [21]:
# l=[0,37,215,249,257,258]
# comments_dataset = comments_dataset.drop(labels=l, axis=0)

# comments_dataset

Here, we have combined the title, subheading and paragraph to compute embeddings and stored it in "text" variable. This variable is further used to compute **embeddings**.

In [22]:
def concatenate_text(examples):
    return {
        "text": examples["title"]
        + " \n "
        + examples["body"]
        + " \n "
        + examples["comments"]
    }


comments_dataset = comments_dataset.map(concatenate_text)

  0%|          | 0/472 [00:00<?, ?ex/s]

In [68]:
comments_dataset

Dataset({
    features: ['title', 'body', 'comments', 'html_url', '__index_level_0__', 'text'],
    num_rows: 472
})

In [69]:
columns = comments_dataset.column_names
columns_to_keep = ["title", "body", "html_url", "comments","text"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
comments_dataset = comments_dataset.remove_columns(columns_to_remove)
comments_dataset

Dataset({
    features: ['title', 'body', 'comments', 'html_url', 'text'],
    num_rows: 472
})

Model selection

In [70]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [71]:
# import torch

# #device = torch.device("cuda")
# model.to(device)

In [72]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

Function to compute embedding values which the model will use further for semantic search

In [73]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    #encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [74]:
embedding = get_embeddings(comments_dataset["text"][0])
embedding.shape

torch.Size([1, 768])

In [75]:
embeddings_dataset = comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

  0%|          | 0/472 [00:00<?, ?ex/s]

In [83]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'body', 'comments', 'html_url', 'text', 'embeddings'],
    num_rows: 472
})

In [97]:
question = ["Therapy for ADHD", "Not able to focus","Child not sleeping properly","bed wetting"]
question_embedding = get_embeddings([question[-1]]).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [98]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)

In [99]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [100]:
for _, row in samples_df.iterrows():
    print(f"TITLE: {row.title}")
    print(f"SCORE: {row.scores}")
    print(f"COMMENT: {row.comments}")
    print(f"URL: {row.html_url}")
    print("=" * 50)
    print()

TITLE: Potty Training: What to Know
SCORE: 45.69811248779297
COMMENT:  Children's Health  P:  Children's Vaccines  P:  Parenting  P:  Pregnancy  P:  More Related Topics  P:  Parenting  P:  P:  P:  P:  P:  P:  P:  Coronavirus Update  P:  Check Your Symptoms  P:  Find A Doctor  P:  Find A Dentist  P:  Connect to Care  P:  Find Lowest Drug Prices  P: Health  A-Z Health A-Z  Health A-Z  Common Conditions   ADD/ADHD  Allergies  Arthritis  Cancer  Coronavirus (COVID-19)  Depression  Diabetes  Eye Health  Heart Disease  Lung Disease  Orthopedics  Pain Management  Sexual Conditions  Skin Problems  Sleep Disorders  View All  Resources   Symptom Checker  WebMD Blogs  Podcasts  Newsletters  Best Hospitals  Find a Doctor  Surgeries and Procedures A-Z Featured Topics Slideshow  Signs You Have Already Had COVID  Slideshow  Get Help for Migraine Relief  P:  ADD/ADHD  P:  Allergies  P:  Arthritis  P:  Cancer  P:  Coronavirus (COVID-19)  P:  Depression  P:  Diabetes  P:  Eye Health  P:  Heart Disease  